In [12]:
import torch
import torch.nn.functional as F
from dataset import get_dataset
from model import CNN
from torch.utils.data.dataloader import DataLoader
from torchvision import transforms
import numpy as np

### set seed and load dataset

In [13]:
torch.manual_seed(2023)
transform = transforms.Compose([
        transforms.ToTensor(),
    ]
    )
train_dataset, val_dataset, test_dataset = get_dataset("faces96", transform=transform)
print(len(train_dataset), len(val_dataset), len(test_dataset))
train_loader = DataLoader(train_dataset, batch_size=len(train_dataset)//2)
val_loader = DataLoader(val_dataset, batch_size=len(val_dataset))
test_loader = DataLoader(val_dataset, batch_size=len(test_dataset))
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = "cpu"

2413 302 301


In [14]:
model = CNN(152, 3, image_size=196).to(device)
lr = 1e-5
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
epoch = 20

for i in range(epoch):
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        logits = model(x)
        loss = F.softmax(x, y)
        loss.backward()
        optimizer.step()
        print(f"epoch {i}, loss {loss.item}")
    for x, y in val_dataset:
        x = x.to(device)
        y = y.to(device)
        predict = model.predict(x)
        acc = torch.sum(x==y)/x.shape[0]
        print(f"epoch {i}, acc {acc}")


RuntimeError: cuDNN error: CUDNN_STATUS_NOT_INITIALIZED